# Calibration p(track same sign|B)

* use 2-folding to calibrate all data
* return all data (tracks), its probs, its calibrated probs

In [ ]:
from scipy.special import logit
from rep.utils import train_test_split
from sklearn import clone

def calibrate_track_prob(estimator, datasets, est_calib, logistic=False, tag_selection=False):
    data = pandas.concat(datasets)
    probs = union(*[estimator.predict_proba(dataset)[:, 1] for dataset in datasets])
    if tag_selection:
        probs = probs[data.tag.values == True]
        data = data[data.tag]
    ind = range(len(probs))
    ind_1, ind_2 = train_test_split(ind)
    est_calib_1, est_calib_2 = clone(est_calib), clone(est_calib)
    probs_1 = probs[ind_1]
    probs_2 = probs[ind_2]
    data.index = ind
    if logistic:
        probs_1 = logit(probs_1)[:, numpy.newaxis]
        probs_2 = logit(probs_2)[:, numpy.newaxis]
        est_calib_1.fit(probs_1, data.ix[ind_1, 'label'])
        est_calib_2.fit(probs_2, data.ix[ind_2, 'label'])        
    else:
        est_calib_1.fit(probs_1, data.ix[ind_1, 'label'], data.ix[ind_1, 'N_sig_sw'])
        est_calib_2.fit(probs_2, data.ix[ind_2, 'label'], data.ix[ind_2, 'N_sig_sw'])
    probs_result = numpy.zeros(len(probs))
    if logistic:
        probs_result[ind_1] = est_calib_2.predict_proba(probs_1)[:, 1]
        probs_result[ind_2] = est_calib_1.predict_proba(probs_2)[:, 1]
    else:
        probs_result[ind_1] = est_calib_2.transform(probs_1)
        probs_result[ind_2] = est_calib_1.transform(probs_2)
    return data, probs, probs_result

# Compute $p(B^+)$

Compute $p(B^+)$ using this representation:

$$ \frac{P(B^+)}{P(B^-)} = \prod_{track} \frac{P(track|B^+)} {P(track|B^-)} = P$$
$$ P(B^+) = \frac {P}{1+P}, $$
where
$$ P(track^+|B^+) = P(\text{track same sign as B}| B) $$
$$ P(track^+|B^-) = 1 - P(\text{track same sign as B}| B)$$
$$ P(track^-|B^+) = 1 - P(\text{track same sign as B}| B) $$
$$ P(track^-|B^-) = P(\text{track same sign as B}| B)$$

In [ ]:
from scipy.special import expit

def compute_B_prob_using_track_prob(data, probs):
    result_event, data_ids = numpy.unique(data.unique, return_inverse=True)
    log_probs = numpy.log(probs) - numpy.log(1 - probs)
    log_probs *= data.signTrack.values
    result_logprob = numpy.bincount(data_ids, weights=log_probs)
    result_label = numpy.bincount(data_ids, weights=data.signB) / numpy.bincount(data_ids)
    result_weight = numpy.bincount(data_ids, weights=data.N_sig_sw) / numpy.bincount(data_ids)
    return result_label, result_weight, expit(result_logprob), result_event

# Compute p(B+) calibrated

* use Isotonic calibration
* randomly divide events into two parts (1-train, 2-calibrate and D2 compute)
* take mean D2 and std

In [ ]:
def calibrate_B_prob(labels, weights, probs, steps=30):
    iso_calibs = []
    aucs = []
    D2 = []
    omega = []
    labels = (labels > 0) * 1
    for step in range(steps):
        train_probs, test_probs, train_labels, test_labels, train_weights, test_weights = train_test_split(
            probs, labels, weights, train_size=0.5)
        iso_est = IsotonicRegression(y_min=0, y_max=1, out_of_bounds='clip')
        iso_est.fit(train_probs, train_labels, train_weights)
        iso_calibs.append(iso_est)
        aucs.append(roc_auc_score(test_labels, test_probs, sample_weight=test_weights))
        probs_calib = iso_est.transform(test_probs)
        alpha = (1 - 2 * (1 - probs_calib))**2
        D2.append(sum(alpha * test_weights) / sum(test_weights))
        omega.append(sum((1 - probs_calib) * test_weights) / sum(test_weights))
    return D2, aucs, iso_calibs, omega